<a href="https://colab.research.google.com/github/Cloud2602/gesture_recognition/blob/main/gesture_recognition_3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Scaricare e aprire acquisizioni.zip
from google.colab import drive
drive.mount('/content/drive')
!unzip -q "/content/drive/MyDrive/acquisizioni.zip" -d "/content/dataset"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%bash
pip install mediapipe

In [32]:
!git clone https://github.com/Cloud2602/gesture_recognition.git
!unzip -q "/content/gesture_recognition/mediapipe_landmarks_two_hands.zip" -d "/content/data"


fatal: destination path 'gesture_recognition' already exists and is not an empty directory.


In [ ]:
import os
import numpy as np
import cv2
import mediapipe as mp
import csv

# MediaPipe setup
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
)

# Percorsi
DATASET_DIR = "/content/3"  # Cambia se necessario
output_npy = "landmarks.npy"
output_labels = "labels.npy"
output_csv = "landmarks.csv"

# Output
all_landmarks = []
all_labels = []

# Mapping label
label_map = {
    "1": 1,
    "2": 2,
    "tre": 3,
    "0": 0
}

# CSV setup
csv_file = open(output_csv, mode='w', newline='')
csv_writer = csv.writer(csv_file)

# Header
header = [f'{dim}{i}' for i in range(21) for dim in ['x', 'y', 'z']]
header.append('label')
csv_writer.writerow(header)

# Contatori
total_images = 0
processed_images = 0
skipped_images = 0

# Scansiona le sottocartelle
for class_name in sorted(os.listdir(DATASET_DIR)):
    print(f"✔️ Cartella: {class_name}")
    class_path = os.path.join(DATASET_DIR, class_name)
    if not os.path.isdir(class_path):
        continue

    if class_name not in label_map:
        print(f"⚠️ La cartella '{class_name}' non ha una label definita. Saltata.")
        continue
    label = label_map[class_name]

    for filename in os.listdir(class_path):
        print(filename)
        if not filename.endswith('_Color.png'):
            continue  # ⛔ Salta file che non terminano in _Color.png

        total_images += 1

        image_path = os.path.join(class_path, filename)
        image = cv2.imread(image_path)
        if image is None:
            print(f"❌ Errore nel caricamento di: {image_path}")
            skipped_images += 1
            continue

        # Converti BGR in RGB per MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)

        if results.multi_hand_landmarks:
            hand = results.multi_hand_landmarks[0]
            landmarks = []
            for lm in hand.landmark:
                landmarks.extend([lm.x, lm.y, lm.z])
            all_landmarks.append(landmarks)
            all_labels.append(label)
            csv_writer.writerow(landmarks + [label])
            processed_images += 1
        else:
            print(f"🙈 Nessuna mano rilevata in: {image_path}")
            skipped_images += 1

# Cleanup
csv_file.close()
hands.close()

np.save(output_npy, np.array(all_landmarks))
np.save(output_labels, np.array(all_labels))

# Report finale
print(f"\n✅ Salvati: {output_npy}, {output_labels}, {output_csv}")
print(f"📊 Totale immagini trovate: {total_images}")
print(f"👌 Immagini elaborate con successo: {processed_images}")
print(f"🚫 Immagini scartate (errore o mano non rilevata): {skipped_images}")


In [40]:
import os
import numpy as np

# === Funzioni di Data Augmentation ===

def flip_landmarks(landmarks):
    landmarks[:, 0] = 1.0 - landmarks[:, 0]
    return landmarks

def zoom_landmarks(landmarks, scale=0.8):
    center = landmarks[0]
    return (landmarks - center) * scale + center

def rotate_landmarks(landmarks, angle_deg=15):
    angle_rad = np.radians(angle_deg)
    center = landmarks[0]
    rotation_matrix = np.array([
        [np.cos(angle_rad), -np.sin(angle_rad)],
        [np.sin(angle_rad),  np.cos(angle_rad)]
    ])
    rotated = np.dot(landmarks[:, :2] - center[:2], rotation_matrix) + center[:2]
    landmarks[:, :2] = rotated
    return landmarks

def translate_landmarks(landmarks, offset=(0.05, -0.03, 0.0)):
    return landmarks + np.array(offset)

# === Percorsi da modificare ===
input_base_dir = "/content/data/mediapipe_landmarks_two_hands"
output_base_dir = "/content/data_augumented"

os.makedirs(output_base_dir, exist_ok=True)

# === Elaborazione dei file ===
for folder_name in sorted(os.listdir(input_base_dir)):
    folder_path = os.path.join(input_base_dir, folder_name)
    print(f"✔️ Cartella: {folder_path}")


    for file_name in os.listdir(folder_path):
            if file_name.endswith('.npy'):
                print(f"✔️ File: {file_name}")
                file_path = os.path.join(folder_path, file_name)
                landmarks = np.load(file_path)
                augmented = []
                for i, landmark in enumerate(landmarks):

                  landmark = landmark.reshape((42, 3))  # Converti in (42, 3)
                  # Lista delle versioni aumentate

                  augmented.append(landmark.copy())

                  # Flip
                  augmented.append(flip_landmarks(landmark.copy()))

                  # Zoom out
                  augmented.append(zoom_landmarks(landmark.copy(), scale=0.8))

                  # Rotazioni
                  augmented.append(rotate_landmarks(landmark.copy(), angle_deg=15))
                  augmented.append(rotate_landmarks(landmark.copy(), angle_deg=-15))

                  # Traslazioni
                  augmented.append(translate_landmarks(landmark.copy(), offset=(0.05, -0.03, 0.0)))
                  augmented.append(translate_landmarks(landmark.copy(), offset=(-0.05, 0.03, 0.0)))
            else:
                print(f"⚠️ File non valido: {file_name}")

    print("augumented lenght: ", len(augmented))
    new_name = f"{os.path.splitext(file_name)[0]}_aug.npy"
    save_path = os.path.join(output_base_dir, new_name)
    np.save(save_path, augmented)
    print(f"✔️ File salvato: {save_path}")



✔️ Cartella: /content/data/mediapipe_landmarks_two_hands/0
⚠️ File non valido: landmarks_0.csv
✔️ File: landmarks_0.npy
augumented lenght:  490
✔️ File salvato: /content/data_augumented/landmarks_0_aug.npy
✔️ Cartella: /content/data/mediapipe_landmarks_two_hands/1
✔️ File: landmarks_1.npy
⚠️ File non valido: landmarks_1.csv
augumented lenght:  490
✔️ File salvato: /content/data_augumented/landmarks_1_aug.npy
✔️ Cartella: /content/data/mediapipe_landmarks_two_hands/2
⚠️ File non valido: landmarks_2.csv
✔️ File: landmarks_2.npy
augumented lenght:  490
✔️ File salvato: /content/data_augumented/landmarks_2_aug.npy
✔️ Cartella: /content/data/mediapipe_landmarks_two_hands/3
⚠️ File non valido: landmarks_3.csv
✔️ File: landmarks_3.npy
augumented lenght:  490
✔️ File salvato: /content/data_augumented/landmarks_3_aug.npy


In [57]:
import numpy

land=numpy.load("/content/gesture_recognition/data_augumented/landmarks_0_aug.npy")
X= land[:][0]
print(len(X))

42
